<a href="https://colab.research.google.com/github/z4n0/LessonsExtractor/blob/main/lessonsExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%ls /content/drive/MyDrive/lessonsExtractor

converted_video.mp4  CSR-20221124.mp4  lessonsExtractor.ipynb  slides/


NB: TURN ON THE T4 GPU

In [3]:
!pip install opencv-python

In [4]:
%cd /content/drive/MyDrive/lessonsExtractor/

/content/drive/MyDrive/lessonsExtractor


In [ ]:
%ls

## conversion from mkv to mp4

In [ ]:
!ffmpeg -i prova.mkv

In [ ]:
#!ffmpeg -i prova.mkv -c:v libx264 -c:a aac prova_converted.mp4

reduce quality and bit rate to help the code run faster

In [ ]:
!ffmpeg -i prova.mkv -s 1280x720 -b:v 1M -c:v libx264 -c:a aac prova_converted.mp4

CODE TO REDUCE QUALITY AND BIT RATE AND SELECT A PIECE OF THE MP4 VIDEO

#check if ffmpeg is compatible for hw accelleration

TURN ON THE T4 GPU!!

A typical successful output snippet might start like this:


In [ ]:

Encoder h264_nvenc [NVIDIA NVENC H.264 encoder]:
    General capabilities: delay hardware
    Threading capabilities: none
    Supported pixel formats: nv12 p010le yuv444p nv16 yuv444p16le bgr0 rgb0 ...
    ...


In [ ]:
#code to check if you can use accelleration
!ffmpeg -h encoder=h264_nvenc

In [ ]:
#check if the tpu is on
!nvidia-smi

Thu Nov 16 17:06:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%ls

converted_video.mp4  CSR-20221124.mp4  lessonsExtractor.ipynb  slides/


In [26]:
input_video_path = '9_Semantic_Segmentation.mp4'

# Extract base name and remove extension to form the new output file name
base_name = os.path.basename(input_video_path)
file_name_without_ext = os.path.splitext(base_name)[0]
output_video_path = f"/content/drive/MyDrive/lessonsExtractor/{file_name_without_ext}_converted.mp4"

resolution = "1280x720"
bitrate = "1M"
start_time = "00:40:00"  # Start at 40 minutes
#duration = "00:30:00"  # Duration of 50 minutes, will stop at the end of the video if it's shorter

# Standard FFmpeg command
#!ffmpeg -ss {start_time} -i {input_video_path} -t {duration} -s {resolution} -b:v {bitrate} -c:v libx264 -c:a aac {output_video_path}

# GPU accelerated version
!ffmpeg -hwaccel cuda -ss {start_time} -i {input_video_path} -s {resolution} -b:v {bitrate} -c:v h264_nvenc -c:a aac {output_video_path}


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

you should see the new converted_video.mp4 here. let's check!

In [27]:
%ls

9_Semantic_Segmentation_converted.mp4  converted_video.mp4  lessonsExtractor.ipynb  slides/
9_Semantic_Segmentation.mp4            CSR-20221124.mp4     output.docx


In [28]:
import cv2
import numpy as np
import os

In [29]:
def save_frame(frame, timestamp, output_folder='slides'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    filename = f"{output_folder}/slide_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"Saved: {filename}")

def frame_difference(prev_frame, curr_frame):
    # Compute the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, curr_frame)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count

def milliseconds_to_minutes_seconds(milliseconds):
    minutes = int(milliseconds / 60000)
    seconds = int((milliseconds % 60000) / 1000)
    return f"{minutes:02d}:{seconds:02d}"

def detect_slides(video_path, threshold=265000, skip_frames=50):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return []

    prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    slide_changes = []

    while cap.isOpened():
        ret, curr_frame = cap.read()
        if not ret:
            break

        if frame_count % skip_frames == 0:
            gray_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            if frame_difference(prev_frame, gray_frame) > threshold:
                timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)  # Get the timestamp in milliseconds
                timestamp_readable = milliseconds_to_minutes_seconds(timestamp_ms)
                slide_changes.append(timestamp_readable)
                save_frame(curr_frame, timestamp_readable)
                prev_frame = gray_frame

        frame_count += 1

    cap.release()
    return slide_changes

input_video_path = output_video_path
slide_changes = detect_slides(input_video_path)
print(f"Slide changes detected at: {slide_changes}")

Saved: slides/slide_00:04.jpg


KeyboardInterrupt: ignored

In [19]:
print(f"detected slides {len(slide_changes)}")

detected slides 3


# Audio extraction

In [10]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2yfuv40l
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2yfuv40l
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=1fa3e8740a77a4a8630e2cda3a6059a90a750146750ebcb293c83243c39d227d
  Stored in directory: /tmp/pip-ephem-wheel-cache-0hntfi17/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [11]:
import whisper

def transcribe_video(video_path, model_size='base'):
    model = whisper.load_model(model_size)
    result = model.transcribe(video_path)
    return result

In [12]:
def interleave_transcription_with_slides(transcription, slide_changes):
    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes) and slide_changes[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

#pdf or docx creation

In [13]:
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.8 MB/s eta 0:00:00


In [20]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

from docx.shared import Inches

from docx import Document
from docx.shared import Inches

def create_document(interleaved_data, video_path, slides_folder='slides'):
    # Extract base name and remove extension to form the DOCX file name
    base_name = os.path.basename(video_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_file = f'{file_name_without_ext}.docx'

    doc = Document()
    for item in interleaved_data:
        if item['type'] == 'slide_change':
            # Add slide image
            slide_image_path = f"{slides_folder}/slide_{item['timestamp']}.jpg"
            p = doc.add_paragraph()
            try:
                p.add_run().add_picture(slide_image_path, width=Inches(6))  # Adjust the width as needed
            except Exception as e:
                p.add_run(f"Could not load slide image: {slide_image_path}")
        else:
            # Add transcribed text
            p = doc.add_paragraph(item['text'])

    doc.save(output_file)
    print(f"Document saved as: {output_file}")


def convert_timestamp_to_seconds(timestamp_str):
    minutes, seconds = map(int, timestamp_str.split(':'))
    return minutes * 60 + seconds

def interleave_transcription_with_slides(transcription, slide_changes):
    # Convert slide change timestamps to seconds
    slide_changes_seconds = [convert_timestamp_to_seconds(ts) for ts in slide_changes]

    interleaved_data = []
    current_slide = 0

    for segment in transcription['segments']:
        # Find the next slide change that occurs after this segment starts
        while current_slide < len(slide_changes_seconds) and slide_changes_seconds[current_slide] <= segment['start']:
            interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
            current_slide += 1

        # Add the transcribed text
        interleaved_data.append({'type': 'transcription', 'timestamp': segment['start'], 'text': segment['text']})

    # Handle any remaining slide changes
    while current_slide < len(slide_changes_seconds):
        interleaved_data.append({'type': 'slide_change', 'timestamp': slide_changes[current_slide]})
        current_slide += 1

    return interleaved_data

# Transcribe video
transcription = transcribe_video(output_video_path)

# Interleave transcription with slide changes
interleaved_data = interleave_transcription_with_slides(transcription, slide_changes)

# Create DOCX document
create_document(interleaved_data, output_video_path)